In [1]:
import tensorflow as tf
import os
os.chdir("..")
os.getcwd()

'd:\\ineuron\\DL_project\\CNN_project'

In [2]:
#model = tf.keras.models.load_model("artifacts/training/model.h5")

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tenoob/CNN_project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="tenoob" 
os.environ["MLFLOW_TRACKING_PASSWORD"]="813fedf452b75e0b297aabb76f145ed142f1a66f"

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [5]:
from cnnProject.constant import *
from cnnProject.utils import read_yaml,create_directories,save_json

In [7]:
class ConfigrationManger:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifact_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= 'artifacts/training/model.h5',
            training_data= 'artifacts/data_ingestion/PetImages',
            mlflow_uri="https://dagshub.com/tenoob/CNN_project.mlflow",
            all_params= self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        
        )

        return eval_config


In [10]:
# training Component
from pathlib import Path
import tensorflow as tf
import mlflow
import mlflow.tensorflow
import urllib.parse as pas

class Evaluation:
    def __init__(self, config: EvaluationConfig) -> None:
        self.config = config

    def _valid_generator(self):
        # will be doing augmentation

        datagenerator_kwargs = dict(rescale=1.0 / 255, validation_split=0.30)

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[
                :-1
            ],  # only takes rows and columns, channels not aka index 0,1
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            # interpolation is a process of determining the unknown values that lie in between the known data points.
            # ex find value between 1 and 2 on a line.
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # validation data
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path):
        return tf.keras.models.load_model(path)

    # start training
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0],
                "accuracy":self.score[1]}

        save_json(path=Path("scores.json"),data = scores)
        #store the scores.json in root for dvc to track it easily

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = pas.urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():

            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metrics(
                {"loss": self.score[0],
                "accuracy":self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")



In [11]:
try:
    config = ConfigrationManger()
    
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

Found 7498 images belonging to 2 classes.
118/118 [==============================] - 1122s 10s/step - loss: 3.2964 - accuracy: 0.7463
INFO:tensorflow:Assets written to: C:\Users\kanan\AppData\Local\Temp\tmpvc2j10x8\model\data\model\assets


2023/02/22 01:14:23 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2022-05-26; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'tensorflow-intel'}
d:\ineuron\DL_project\CNN_project\env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}